# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기
---
## Face Detection
- 이미지나 동영상에서 특징을 찾아서 얼굴을 찾는다.

## Face Recognition
- 얼굴 인식을 통해 누구인지 알 수 있다.

패키지 설치
> pip install mediapipe  

오류 날 때
> pip install --user mediapipe

## 기본 코드 정리
---

In [1]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 동영상 파일 열기
cap = cv2.VideoCapture("face_video.mp4")


with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        # BGR 값을 RBG 값으로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 얼굴을 검출하여 결과로 반환
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        # RBG를 BGR로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 검출 된 얼굴이 있다면
        if results.detections:
             # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                # 반복문을 통해서 얼굴에 네모나 점등 표시한다.
                mp_drawing.draw_detection(image, detection)
                
        # 캠에서 보여주면 좌우 반전을 해야 보기 편하지만 동영상을 사용하기 때문에 Flip은 안써도 된다.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        if cv2.waitKey(1) == ord("q"):
            break
            
cap.release()
cv2.destroyAllWindows()

- 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
> mp.solutions.face_detection = 얼굴 검출을 위한 face_detection 모듈 사용  
> mp.solutions.drawing_utils = 얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용  

- FaceDetection 함수 정의
> FaceDetection(model_selection(카메라로 부터 거리), min_detection_confidence(어느 정도의 퍼센트 이상되면 얼굴로 인식))
>> model_selection = 0 : 카메라로 부터 2m 이내 근거리에 적합
>> model_selection = 1 : 카메라로 부터 5m 이내 적합

## 눈과 코 인식하여 도형 그리기
---

In [2]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 동영상 파일 열기
cap = cv2.VideoCapture("face_video.mp4")


with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        # BGR 값을 RBG 값으로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 얼굴을 검출하여 결과로 반환
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        # RBG를 BGR로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 검출 된 얼굴이 있다면
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                # 반복문을 통해서 얼굴에 네모나 점등 표시한다.
                # mp_drawing.draw_detection(image, detection)
                # print(detection)

                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose = keypoints[2] # 코 끝부분
                
                h, w, _ = image.shape # height, width, channel : 이미지로 부터 세로 가로 크기 가져옴
                
                # 이미지 내에서 실제 좌표(x, y)
                right_eye = (int(right_eye.x * w), int(right_eye.y * h))
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose = (int(nose.x * w), int(nose.y * h))
                
                # 양 눈 동그라미 그리기
                cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
                cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)
                cv2.circle(image, nose, 75, (0, 255, 255), 10, cv2.LINE_AA)
                
                
        # 켐에서 보여주면 좌우 반전을 해야 보기 편하지만 동영상을 사용하기 때문에 Flip은 안써도 된다.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        if cv2.waitKey(1) == ord("q"):
            break
            
cap.release()
cv2.destroyAllWindows()

#### print(detection) 를 통하여 정보를 알 수 있다
---
label_id: 0  
score: 0.972063422203064 - 신뢰도(min_detection_confidence)

- 위치 좌표(전체 이미지 기준(최대 1)  
location_data {
  format: RELATIVE_BOUNDING_BOX  
  relative_bounding_box {  
    xmin: 0.34010049700737  
    ymin: 0.27738773822784424  
    width: 0.16470429301261902  
    height: 0.2928076982498169  
  }  
    
  - 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
  relative_keypoints {
    x: 0.39748260378837585
    y: 0.3514038920402527
  }
  relative_keypoints {
    x: 0.472693532705307
    y: 0.3491246700286865
  }
  relative_keypoints {
    x: 0.4475392699241638
    y: 0.41251301765441895
  }
  relative_keypoints {
    x: 0.4445818364620209
    y: 0.4767593741416931
  }
  relative_keypoints {
    x: 0.3378288745880127
    y: 0.39561253786087036
  }
  relative_keypoints {
    x: 0.49736836552619934
    y: 0.38518768548965454
  }
}

### 위치 좌표 찾기
- 이미지의 크기에 따라 x, y 좌표를 곱해줘야 한다.

## 그림판 이미지 씌우기
---

In [3]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 동영상 파일 열기
cap = cv2.VideoCapture("face_video.mp4")

# 이미지 불러오기(가로 x 세로)
image_right_eye = cv2.imread("right_eye.png") # 100 x 100
image_left_eye = cv2.imread("left_eye.png") # 100 x 100
image_nose = cv2.imread("nose.png") # 300 x 100

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        # BGR 값을 RBG 값으로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 얼굴을 검출하여 결과로 반환
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        # RBG를 BGR로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 검출 된 얼굴이 있다면
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                # 반복문을 통해서 얼굴에 네모나 점등 표시한다.
                # mp_drawing.draw_detection(image, detection)
                # print(detection)

                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose = keypoints[2] # 코 끝부분
                
                h, w, _ = image.shape # height, width, channel : 이미지로 부터 세로 가로 크기 가져옴
                
                # 이미지 내에서 실제 좌표(x, y)
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) - 20, int(left_eye.y * h) - 100)
                nose = (int(nose.x * w), int(nose.y * h))
                
                # 각 특징에 이미지 그리기
                image[right_eye[1] - 50:right_eye[1] + 50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                image[left_eye[1] - 50:left_eye[1] + 50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                image[nose[1] - 50:nose[1] + 50, nose[0]-150:nose[0]+150] = image_nose
                
                
        # 켐에서 보여주면 좌우 반전을 해야 보기 편하지만 동영상을 사용하기 때문에 Flip은 안써도 된다.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        if cv2.waitKey(1) == ord("q"):
            break
            
cap.release()
cv2.destroyAllWindows()

### 위치 좌표
- 각 특징은 중간 점이므로 가로, 세로 크기의 절반 값을 +/- 를 해주었다.
- 값을 모를 때
> image.shape를 통해서 할 수 있다.

## 캐릭터 이미지 씌우기
---

In [4]:
import cv2
im = cv2.imread("cat_left_ear.png")
im.shape

(100, 100, 3)

> channel 값이 3이라고 나온다 -> 3이면 BGR값만 가져온 것이다.  
옵션을 적용해 주지않으면 투명도가 무시된다.

In [5]:
import cv2
im = cv2.imread("cat_nose.png", cv2.IMREAD_UNCHANGED)
im.shape

(100, 300, 4)

In [8]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 동영상 파일 열기
cap = cv2.VideoCapture("face_video.mp4")

# 이미지 불러오기(가로 x 세로)
image_right_eye = cv2.imread("cat_right_ear.png") # 100 x 100
image_left_eye = cv2.imread("cat_left_ear.png") # 100 x 100
image_nose = cv2.imread("cat_nose.png") # 300 x 100

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        # BGR 값을 RBG 값으로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 얼굴을 검출하여 결과로 반환
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        # RBG를 BGR로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 검출 된 얼굴이 있다면
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                # 반복문을 통해서 얼굴에 네모나 점등 표시한다.
                # mp_drawing.draw_detection(image, detection)
                # print(detection)

                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose = keypoints[2] # 코 끝부분
                
                h, w, _ = image.shape # height, width, channel : 이미지로 부터 세로 가로 크기 가져옴
                
                # 이미지 내에서 실제 좌표(x, y)
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) - 20, int(left_eye.y * h) - 100)
                nose = (int(nose.x * w), int(nose.y * h))
                
                # 각 특징에 이미지 그리기
                image[right_eye[1] - 50:right_eye[1] + 50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                image[left_eye[1] - 50:left_eye[1] + 50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                image[nose[1] - 50:nose[1] + 50, nose[0]-150:nose[0]+150] = image_nose
                
                
        # 켐에서 보여주면 좌우 반전을 해야 보기 편하지만 동영상을 사용하기 때문에 Flip은 안써도 된다.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        if cv2.waitKey(1) == ord("q"):
            break
            
cap.release()
cv2.destroyAllWindows()

- cv2.IMREAD_UNCHANGED하여 파일을 읽는다면
> 대상 동영상은 3채널이고 이미지는 4채널이기 떄문에 오류가 발생 

- cv2.IMREAD_UNCHANGED 넣지 않을 경우
> 투명도가 무시되어 코드는 실행되지만 뒤에 배경을 지울 수가 없음

### 이미지 channel이 4인 경우

In [7]:
import cv2
import mediapipe as mp

# 대상 이미지(3채널), x, y좌표, width, height, 덮어씌울 이미지(4채널)
def overlay(img, x, y, w, h, overlay_image):
    alpha = overlay_image[:, :, 3]
    mask_image = alpha / 255 # 0 ~ 1 사이의 값
    
    # channel BRG
    for c in range(0, 3):
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 동영상 파일 열기
cap = cv2.VideoCapture("face_video.mp4")

# 이미지 불러오기(가로 x 세로)
image_right_eye = cv2.imread("cat_right_ear.png", cv2.IMREAD_UNCHANGED) # 100 x 100
image_left_eye = cv2.imread("cat_left_ear.png", cv2.IMREAD_UNCHANGED) # 100 x 100
image_nose = cv2.imread("cat_nose.png", cv2.IMREAD_UNCHANGED) # 300 x 100

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        # BGR 값을 RBG 값으로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 얼굴을 검출하여 결과로 반환
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        # RBG를 BGR로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 검출 된 얼굴이 있다면
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                # 반복문을 통해서 얼굴에 네모나 점등 표시한다.
                # mp_drawing.draw_detection(image, detection)
                # print(detection)

                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose = keypoints[2] # 코 끝부분
                
                h, w, _ = image.shape # height, width, channel : 이미지로 부터 세로 가로 크기 가져옴
                
                # 이미지 내에서 실제 좌표(x, y)
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) - 20, int(left_eye.y * h) - 100)
                nose = (int(nose.x * w), int(nose.y * h))
                
                # 각 특징에 이미지 그리기
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose, 150, 50, image_nose)
                
        # 켐에서 보여주면 좌우 반전을 해야 보기 편하지만 동영상을 사용하기 때문에 Flip은 안써도 된다.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        if cv2.waitKey(1) == ord("q"):
            break
            
cap.release()
cv2.destroyAllWindows()

### 코드 분석

- overlay(대상 이미지(3채널), x, y좌표, width, height, 덮어씌울 이미지(4채널))
> alpha = overlay_image[:, :, 3] : BGRA 중 Alpha 값만 가져온다  
> mask_image = alpha / 255  
>> mask_image는 0 ~ 1 값을 가진다 (1 : 불투명, 0: 완전 투명한 이미지)  
> (1 - mask_image)는 1 ~ 0 값을 가진다  
> 두 값을 더하면 항상 1 == 완전 불투명를 의미한다.

 - 반복문  
 image[세로영역, 가로영역, 채널] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

> 결과 이미지 3채널(BGR) =  
{덮어씌울 이미지 4채널(BGRA 중 BGR) X mask_image(투명도)} + {대상이미지 3채널(BGR) X (1 - mask_image)}  

- ex1)  
> ???              = {노랑(0, 255, 255) x 1.0} + {빨강(0, 0, 255) x 0.0}  
{노랑(0, 255, 255) = {노랑(0, 255, 255)        +     (0, 0, 0)
- ex2)
>???              = {노랑(0, 255, 255) x 0.0} + {빨강(0, 0, 255) x 1.0}  
{노랑(0, 255, 255) =  (0, 0, 0)  +   {빨강(0, 0, 255) x 1.0}  
- ex3)
>???              = {노랑(0, 255, 255) x 0.4} + {빨강(0, 0, 255) x 0.6}  
주황(0, 102, 255) = [(0, 255, 255) * 0.4] = [(0, 102, 102) * 0.6] + (0, 0, 255)]= (0, 0, 153)

#### 덮어 씌울 이미지에 투명도를 곱하고, 대상 이미지에 (1 - 투명도)를 곱하고 두 이미지를 섞어서 적절한 값을 도출한다.

## 참고
---
[코드 참고](https://google.github.io/mediapipe/solutions/face_detection)  
[동영상](https://www.pexels.com/video/a-man-having-a-discussion-with-a-woman-3256542/)  
[캐릭터 이미지](https://www.freepik.com/free-vector/cute-animal-masks-video-chat-application-effect-filters-set_6380101.htm)  
[무료 이미지 편집 도구]( https://pixlr.com/kr/
)  
[무료 이미지 참고](https://pixabay.com/)  
[학습 자료](https://opencv-python.readthedocs.io/en/latest/)